In [ ]:
# import
import pandas as pd
import numpy as np

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

from matplotlib.lines import Line2D
import matplotlib.dates as mdates
myFmt = mdates.DateFormatter('%d %b')
import datetime

In [ ]:
!pip install plotly

In [ ]:
# create function for metrics and visualization
def metrics(true, pred):
    cm = confusion_matrix(true, pred)
    tn = cm[0][0]
    fp = cm[0][1]
    fn = cm[1][0]
    tp = cm[1][1]

    precision = float(tp/(tp+fp)) 
    recall = float(tp/(tp+fn))  
    f1 = float((2*precision*recall)/(precision+recall))
    tss = float((tp/(tp+fn)) - (fp/(fp+tn)))
    
    return precision, recall, f1, tss, tn, fp, fn, tp

def create_conditions(true, pred):
    return [(true == 1) & (pred == 1),
     (true == 0) & (pred == 0),
     (true == 1) & (pred == 0),
     (true == 0) & (pred == 1)]

def add_text_plot():
    legend_elements = [Line2D([0], [0], marker='o', color='w', label='TN', markerfacecolor="#1f77b4", markersize=6),
                       Line2D([0], [0], marker='o', color='w', label='FP', markerfacecolor='#ff7f0e', markersize=6),
                       Line2D([0], [0], marker='o', color='w', label='TP', markerfacecolor='#2ca02c', markersize=6),
                       Line2D([0], [0], marker='o', color='w', label='FN', markerfacecolor='#d62728', markersize=6)]
    plt.legend(handles=legend_elements, loc='upper right')
    plt.ylabel(r'$\sigma_\phi [rad]$', fontsize=12)
    plt.axhline(0.1, color='silver', linewidth=2)
    plt.grid(True, linewidth=0.3)
    plt.ylim(0, 1)
    plt.gca().xaxis.set_major_formatter(myFmt)

## Data loading

In [ ]:
autoreg_15_df = pd.read_csv("post_processing/autoreg15_predictions_on_test-data_with_time-bin_std-cov_full-test.csv").set_index("index_time_bin")
PC_df = pd.read_csv("post_processing/PC_predictions_on_test-data_with_time-bin_std-cov_full-test.csv").set_index("index_time_bin")
all_parameters_df = pd.read_csv("post_processing/all_parameters_predictions_on_test-data_with_time-bin_std-cov_full-test.csv").set_index("index_time_bin")

In [ ]:
# create intersection
models_dfs = pd.concat([autoreg_15_df, PC_df, all_parameters_df], axis=1)
models_dfs = models_dfs.dropna()
models_dfs = models_dfs.rename(columns={"autoreg_15_test": "y_test", "autoreg_15_time": "time_index",
                                          "autoreg_15_bin": "bin_index"})
models_dfs = models_dfs[["time_index", "bin_index", "y_test",
                         "autoreg_15_pred", "autoreg_15_cov_pred", "autoreg_15_cov_test",
                           "PC_pred", "PC_cov_pred", "PC_cov_test",
                           "all_parameters_pred", "all_parameters_cov_pred", "all_parameters_cov_test"]]
models_dfs

In [ ]:
# create persistence prediction
models_dfs['persistence_shift_time'] = pd.to_datetime(models_dfs.time_index) - datetime.timedelta(minutes=15)
models_dfs['persistence_shift_time'] = models_dfs['persistence_shift_time'].astype(str)
models_dfs['persistence_shift_time'] = models_dfs['persistence_shift_time']+" "+models_dfs["bin_index"].astype(str)
models_dfs

In [ ]:
models_dfs['index_time_bin'] = models_dfs.index
mapping_true = dict(models_dfs[['index_time_bin', "y_test"]].values)
models_dfs['persistence'] = models_dfs["persistence_shift_time"].map(mapping_true)
models_dfs

In [ ]:
models_dfs["persistance_true"]=np.where(models_dfs["y_test"] >= 0.1, 1, 0)
models_dfs["persistance_pred"]=np.where(models_dfs["persistence"] >= 0.1, 1, 0)

In [ ]:
#autoreg15
varcoef_true_set1 = 0.5
varcoef_pred_set1 = 0.01
#PC
varcoef_true_set2 = 0.5
varcoef_pred_set2 = 0.1
#all_parameters
varcoef_true_set3 = 0.5
varcoef_pred_set3 = 0.05


In [ ]:
# select bin
scinti_df = models_dfs[models_dfs["bin_index"]==13502].copy()
scinti_df = scinti_df.set_index(pd.to_datetime(scinti_df["time_index"]))

In [ ]:
plt.rcParams["figure.figsize"] = (10,10)

scinti_df["True_set1"] = np.where(scinti_df["autoreg_15_cov_test"] < varcoef_true_set1, 0, 1)
scinti_df["True_set2"] = np.where(scinti_df["PC_cov_test"] < varcoef_true_set2, 0, 1)
scinti_df["True_set3"] = np.where(scinti_df["all_parameters_cov_test"] < varcoef_true_set3, 0, 1)
scinti_df["True_set4"] = scinti_df["persistance_true"]

scinti_df["Pred_set1"] = np.where(scinti_df["autoreg_15_cov_pred"] < varcoef_pred_set1, 0, 1)
scinti_df["Pred_set2"] = np.where(scinti_df["PC_cov_pred"] < varcoef_pred_set2, 0, 1)
scinti_df["Pred_set3"] = np.where(scinti_df["all_parameters_cov_pred"] < varcoef_pred_set3, 0, 1)
scinti_df["Pred_set4"] = scinti_df["persistance_pred"]

values = [3, 1, 4, 2]

conditions_1 = create_conditions(scinti_df['True_set1'], scinti_df['Pred_set1'])
scinti_df['e1_cm_color'] = np.select(conditions_1, values)
conditions_2 = create_conditions(scinti_df['True_set2'], scinti_df['Pred_set2'])
scinti_df['e2_cm_color'] = np.select(conditions_2, values)
conditions_3 = create_conditions(scinti_df['True_set3'], scinti_df['Pred_set3'])
scinti_df['e3_cm_color'] = np.select(conditions_3, values)
conditions_4 = create_conditions(scinti_df['True_set4'], scinti_df['Pred_set4'])
scinti_df['e4_cm_color'] = np.select(conditions_4, values)

precision1, recall1, f11, tss1, tn1, fp1, fn1, tp1 = metrics(scinti_df["True_set1"], scinti_df["Pred_set1"])
precision2, recall2, f12, tss2, tn2, fp2, fn2, tp2 = metrics(scinti_df["True_set2"], scinti_df["Pred_set2"])
precision3, recall3, f13, tss3, tn3, fp3, fn3, tp3 = metrics(scinti_df["True_set3"], scinti_df["Pred_set3"])
precision4, recall4, f14, tss4, tn4, fp4, fn4, tp4 = metrics(scinti_df["True_set4"], scinti_df["Pred_set4"])

size_of_marker=2.5


ix=0
part_df = scinti_df
# plot persistence, autoreg15, PC, all_parameters predictions
for i in range(0,52):  
    fig, ax = plt.subplots(4,1)
    plt.setp(ax, xlim=(pd.Timestamp('2019-01-01 00:00:00')+datetime.timedelta(days=i*7), pd.Timestamp('2019-01-01 00:00:00')+datetime.timedelta(days=i*7+7)))
    plt.suptitle('Autoregressive models predictions, bin 13725', fontsize=14)

    # set1
    plt.subplot(4, 1, 1).set_title("Persistence", fontsize=12)
    scatter1=plt.scatter(part_df.index, part_df["y_test"], s=size_of_marker, c=part_df['e4_cm_color'], cmap="tab10", vmin=1, vmax=10)
    add_text_plot()
    
    # set2
    plt.subplot(4, 1, 2).set_title("Autoregressive", fontsize=12)
    scatter2=plt.scatter(part_df.index, part_df["y_test"], s=size_of_marker, c=part_df['e1_cm_color'], cmap="tab10", vmin=1, vmax=10)
    add_text_plot()
    
    # set3
    plt.subplot(4, 1, 3).set_title("Base Multivariate PC", fontsize=12)
    scatter3=plt.scatter(part_df.index, part_df["y_test"], s=size_of_marker, c=part_df['e2_cm_color'], cmap="tab10", vmin=1, vmax=10)
    add_text_plot()
    
    # set4
    plt.subplot(4, 1, 4).set_title("Extended Multivariate Ap, Asy/H, Bz, PC", fontsize=12)
    scatter4=plt.scatter(part_df.index, part_df["y_test"], s=size_of_marker, c=part_df['e3_cm_color'], cmap="tab10", vmin=1, vmax=10)
    add_text_plot()
    
    plt.subplots_adjust(left=0.1,
                    bottom=0.1,
                    right=0.9,
                    top=0.9,
                    wspace=0.4,
                    hspace=0.4)
    #plt.tight_layout()
    #plt.savefig(f"img/30_v2/img{ix}.png")
    plt.show()
    
    ix=ix+1